# A* search - 8 Puzzle

## Introduction

A* search is the first step to introduce informed search.

The first thing one has to do is to build an admissable heuristic function, that must always be an underestimation of the real cost.

In [1]:
from collections import deque
import itertools
import math
import heapq
import time
import random
from colorama import Fore, Style
import numpy as np

After some **import** we define how large our puzzle has to be and we construct our winning board by the command `arange`, adding a zero to his end by `append`. Our initial board will be a shuffled winning board. `tolist()` simply convert an array to a list. 

Pay attention when copying lists or arrays! If you *don't* put `[:]` then they will be treated as the *same* object, so you cannot modify one without changing the other.

Furthermore we ask the code to trigger an error if our puzzle is  not a square!

In [8]:
Puzzle=8

WINNING_BOARD = (np.append(np.arange(1,Puzzle+1,1),0)).tolist()
INITIAL_BOARD = WINNING_BOARD[:]
random.shuffle(INITIAL_BOARD)

if math.sqrt(Puzzle+1)%1==0:
    pass
else:
    raise ValueError("Provide a value for the puzzle that is equal to the number of elements of a square matrix: sqrt(n+1) must be integer.")

The following function tells us if the 8 puzzle is **unsolvable**. With a 15 puzzle we need a different function.

This function counts the number of the couples of successive tiles that are inverted with respect to their goal configuration. A puzzle of this kind is solvable if there is an even number of couples that are inverted.

From Tralli, sec. 1.2.1.2: 4In a 8-puzzle the invariant is the parity of the inversions. A pair of tiles is said to form an inversion if the values on the tiles are in reverse order with respect to their appearance in the goal state. In other words, we can linearize the sequence of tiles (ignoring the blank space) and two tiles a and b form an inversion if a precedes b in the sequence, but a > b (practically we count how many lower numbers a tile precedes). It is easy to see that the parity of the inversion is invariant. In fact, when we slide a tile, we either make a row move or a column move: a row move doesn’t change the inversion count, whereas a column move can change it of 􀀀2, 2 or 0. As general rule, it follows that is not possible to solve an instance of 8-puzzle if the number of inversions is odd in the input state, while it is solvable if the number of inversions is even.

In [9]:
def isunsolvable(board,dim): # dim is the dimension of the puzzle
    count=0
    for i in range(dim):
        for j in range(i,dim+1):
            if (board[i] > board[j] and              
                board[j]!=0):
                count+=1
    if count%2==1:
        return True
    else:
        return False

This while cycle shuffle our puzzle until it is solvable

In [11]:
# I suppose they built a function of is-not type to make this while True loop, else they just plain crazy
while isunsolvable(INITIAL_BOARD,Puzzle):
      random.shuffle(INITIAL_BOARD)

Below some useful variables and just a function that print our board in a nice way:

In [12]:
LEN_BOARD = Puzzle+1 #we have to count on the 0

LEN = int(math.sqrt(LEN_BOARD))

def board_print(board):
    print("\n")
    for i in range(LEN):
        print(board[i * LEN: (i + 1) * LEN])


<br>  

So we have to go from

In [6]:
board_print(INITIAL_BOARD)



[0, 6, 1]
[4, 5, 8]
[3, 7, 2]


<br>
to

In [7]:
board_print(WINNING_BOARD)



[1, 2, 3]
[4, 5, 6]
[7, 8, 0]


<br>

## Classes and Functions

here the class `Puzzle` is defined with some functions:
<br>
<br>
`is_win` returns the winning board when called
<br>
<br>

`manhattan` returns the distance in steps between the starting board and the goal board. Here **i** is the index of  a tile in the initial board so `self.board[i]` returns the numeber there located. `WINNING_BOARD.index()` is the goal index, where the number `self.board[i]` in position *i* has to arrive.
<br>
The distance is here calculated based on the steps. % is the modulus operation, // is the integer division. If our matrix has indices from 0 to 8 the modulus operation gives us

$$\begin{pmatrix} 0 & 1 & 2 \\ 3 & 4 & 5 \\ 6 & 7 & 8 \\ \end{pmatrix} \ \text{mod } 3\,\,=\,\,\begin{pmatrix} 0 & 1 & 2 \\ 0 & 1 & 2 \\ 0 & 1 & 2 \\ \end{pmatrix} $$ 

so the column index. The result of integer division gives the row index

$$\begin{pmatrix} 0 & 1 & 2 \\ 3 & 4 & 5 \\ 6 & 7 & 8 \\ \end{pmatrix} \ \text{div } 3\,\,=\,\,\begin{pmatrix} 0 & 0 & 0 \\ 1 & 1 & 1 \\ 2 & 2 & 2 \\ \end{pmatrix} $$ 


<br>
<br>

`misplaced` returns simply as a distance the number of misplaced numbers

<br>
<br>

In [8]:
class Puzzle:
    def __init__(self, board,parent, depth):
        self.board = board
        self.parent = parent
        self.depth = depth + 1 # depth is the number of times the object has been initiated, and increases at each run

        if MANHATTAN:
            self.value = self.manhattan() + self.depth

        elif MISPLACED:
            self.value = self.misplaced() + self.depth

    def is_win(self):
        return self.board == WINNING_BOARD

    def manhattan(self):
        counter = 0
        for i in range(LEN_BOARD):
            index = WINNING_BOARD.index(self.board[i])

            counter += abs((i % LEN - index % LEN)) + abs((i // LEN - index // LEN))

        return counter

    def misplaced(self):
        counter = 0
        for i in range(LEN_BOARD):
            if self.board[i] != WINNING_BOARD[i] and self.board[i] != 0:
                counter += 1

        return counter


<br>
Without going in too much details:
<br>

`get_possible_moves` receive in argument the current board and tells, as a list, the possible moves.
<br>

`get_board` receive in argument the current board and the possible moves.
<br>
<br>
<br>

`bfs` do all the job. It uses `heapq` and `deque` which we have imported at the start of this notebook.

`deque` is a double-ended queue, or deque, it has the feature of adding and removing elements both from left and right. In particular `queque.popleft()` remove and element from the left.


`heapq` module provides an implementation of the heap queue algorithm, also known as the priority queue algorithm. Heaps are binary trees for which every parent node has a value less than or equal to any of its children. 

Of `heapq` module we only to use `heapq.heappush()`, `heapq.heappop()`. The first push an item into the heap list, the second pop the smallest item instead
<br>

In [9]:
def get_possible_moves(board):
    index = board.index(0)
    moves = []

    if int(index % LEN) > 0:
        moves.append((index, index - 1))

    if int(index % LEN) < LEN - 1:
        moves.append((index, index + 1))

    if int(index // LEN) > 0:
        moves.append((index, index - LEN))

    if int(index // LEN) < LEN - 1:
        moves.append((index, index + LEN))

    return moves


def get_board(board, move):
    new_board = board.board[:]

    new_board[move[0]] = new_board[move[1]]
    new_board[move[1]] = 0

    return Puzzle(new_board, board, board.depth)


def list_to_string(ls):
    return "".join(str(ls))


def bfs(node):
    queue = deque([node])
    heap = []
    counter = itertools.count()

    if MANHATTAN or MISPLACED:
        heapq.heappush(heap, (node.value, next(counter), node))

    visited = set()
    k = 0

    while queue:

        if MANHATTAN or MISPLACED:
            pop = heapq.heappop(heap)[2]
        else:
            pop = queue.popleft()

        if k % 10000 == 0 and k !=0:
            if MANHATTAN or MISPLACED:
                print('The heap length is ',len(heap))
            else:
                print('The queque length is ',len(queue))

        if pop.is_win():
            if MANHATTAN or MISPLACED:
                print('The FINAL heap length is ',len(heap))
            else:
                print('The FINAL queque length is ',len(queue))
            return pop

        for x in get_possible_moves(pop.board):
            node_ = get_board(pop, x)

            if list_to_string(node_.board) not in visited:
                visited.add(list_to_string(node_.board))
                if MANHATTAN or MISPLACED:
                    heapq.heappush(heap, [node_.value, next(counter), node_])
                else:
                    queue.append(node_)
        k += 1

## Let's start

Here our program finally starts. Before we decide which method we want to use in our search

In [10]:
# Choose the strategy you want (one xor the other)
MANHATTAN = True
MISPLACED = False

In [13]:
start_time = time.time()

root = Puzzle(INITIAL_BOARD, None, 0)
winner = bfs(root)

a = 0

while winner is not None:
    a += 1
    board_print(winner.board)
    winner = winner.parent


print("--- %s seconds ---" % (time.time() - start_time))
print("{} Moves.".format(a - 1))

The FINAL heap length is  286


[1, 2, 3]
[4, 5, 6]
[7, 8, 0]


[1, 2, 3]
[4, 5, 0]
[7, 8, 6]


[1, 2, 3]
[4, 0, 5]
[7, 8, 6]


[1, 0, 3]
[4, 2, 5]
[7, 8, 6]


[1, 3, 0]
[4, 2, 5]
[7, 8, 6]


[1, 3, 5]
[4, 2, 0]
[7, 8, 6]


[1, 3, 5]
[4, 2, 6]
[7, 8, 0]


[1, 3, 5]
[4, 2, 6]
[7, 0, 8]


[1, 3, 5]
[4, 0, 6]
[7, 2, 8]


[1, 0, 5]
[4, 3, 6]
[7, 2, 8]


[0, 1, 5]
[4, 3, 6]
[7, 2, 8]


[4, 1, 5]
[0, 3, 6]
[7, 2, 8]


[4, 1, 5]
[3, 0, 6]
[7, 2, 8]


[4, 1, 5]
[3, 6, 0]
[7, 2, 8]


[4, 1, 0]
[3, 6, 5]
[7, 2, 8]


[4, 0, 1]
[3, 6, 5]
[7, 2, 8]


[4, 6, 1]
[3, 0, 5]
[7, 2, 8]


[4, 6, 1]
[3, 5, 0]
[7, 2, 8]


[4, 6, 1]
[3, 5, 8]
[7, 2, 0]


[4, 6, 1]
[3, 5, 8]
[7, 0, 2]


[4, 6, 1]
[3, 5, 8]
[0, 7, 2]


[4, 6, 1]
[0, 5, 8]
[3, 7, 2]


[0, 6, 1]
[4, 5, 8]
[3, 7, 2]
--- 0.05396842956542969 seconds ---
22 Moves.
